In [ ]:
# written by Maximilian Ullherr, maximilian.ullherr@physik.uni-wuerzburg.de,
# Lehrstuhl fuer Roentgenmikroskopie/Universitaet Wuerzburg, Josef-Martin-Weg 63, 97074 Wuerzburg, Germany
# last updated 2019-04-26

%pylab
import os
save = True
        
print('kernel ready')
os.makedirs('fig', exist_ok=True)

# 2.1 Image Quality Measures: fig 2

In [ ]:
# the physical data files must be in the following format: 
# first column: energy in eV, second column: absorption in 1/cm
mu_Al = genfromtxt("physical_data/Al_mu_cm.txt", unpack=True, skip_header=1)
mu_Fe = genfromtxt("physical_data/Fe_mu_cm.txt", unpack=True, skip_header=1)
mu_W = genfromtxt("physical_data/W_mu_cm.txt", unpack=True, skip_header=1)

for mu in (mu_Al, mu_Fe, mu_W):  # scale energy from eV to keV
    E = mu[0]
    E /= 1e3

fig, ax = plt.subplots(1, 1, figsize=(5, 3))
fig.set_tight_layout(True)

ax.loglog(*mu_Al)
ax.loglog(*mu_Fe)
ax.loglog(*mu_W)
ax.grid()
ax.set_ylabel('$\mu$ [1/cm]')
ax.set_xlabel('energy [keV]')
ax.legend(['aluminum (Al, Z=13)', 'iron (Fe, Z=26)', 'tungsten (W, Z=74)'])
ticks = (1, 3, 6, 10, 30, 60, 100)
ticklabels = [f'{val}' for val in ticks]
ax.set_xticks(ticks)
ax.set_xticklabels(ticklabels)
ax.set_xlim(2.9, 200)
display(fig)
plt.close(fig)
if save:
    fig.savefig('fig/mu_examples.png')
    fig.savefig('fig/mu_examples.pdf')
print(os.path.abspath('fig/mu_examples.pdf'))

In [ ]:
from scipy.interpolate import interp1d

def curve_at(pos, coord, func):
    return interp1d(coord, func)(pos)

print(curve_at(60, *mu_Fe)/curve_at(30, *mu_Fe), 2**-3)
print(4.20356E-007/1.69159E-006, 2**-2)

# 3.5: optimal computational energy weigthing, fig. 3 (top)

In [ ]:
I_1 = 1
I_2 = 1
mu_1 = 1
w = logspace(-1.6, 0.3, 5000)

fig, ax = plt.subplots(1, 1, figsize=(5, 3.))
fig.set_tight_layout(True)

for k, mu_2 in enumerate([0.14, 0.25, (sqrt(2)-1), .71, 1][::-1]):
    SNR_sum = (I_1*mu_1 + I_2*mu_2*w)**2/(I_1 + I_2*w**2)
    w_opt = w[argmax(SNR_sum)]
    assert isclose(w_opt, mu_2, atol=1e-2)
    ax.semilogx(w, SNR_sum, ls=('-' if k%2 == 0 else '--'),
                 label=(f'$\kappa_2$ = {mu_2:.2f}$\,\kappa_1$' if mu_2 != 1 else f'$\kappa_2$ = $\kappa_1$'))
                
ax.semilogx(w, (w**2+mu_1**2), 'black', ls=':', label='SNR$_{max}(\kappa_2 = w_2/w_1)$')

#ax.semilogx(w, (w+mu_1)**2/2, 'grey', ls='-.', label='SNR$(\kappa_2, w_1=w_2)$')
    
ax.set_ylabel('SNR$_{1+2}(u_a)$')
ax.set_xlabel('CEW weight $w_2/w_1$')
ax.grid(which='both')
ax.legend()#loc='lower left')
ax.set_xscale('log')#, subsx=(2, 4, 6, 8))
ticks = (0.03, 0.06, 0.1, 0.3, 0.6, 1)
ax.set_xticks(ticks)
ax.set_xticklabels(['{:.4g}'.format(x) for x in ticks])
ax.set_xlim(w.min(), w.max())
ax.set_ylim(0.45, 2.6)
                
                
arrowprops = dict(width=0.5, headwidth=4, headlength=4, color='black', shrink=0.02)
                
ax.annotate('a)', xy=(1, 0.63), xytext=(0.8, 0.5), arrowprops=arrowprops)  
ax.annotate('b)', xy=(1, 0.78), xytext=(0.8, 0.9), arrowprops=arrowprops)   
ax.annotate('c)', xy=(1, 1), xytext=(0.8, 1.15), arrowprops=arrowprops)  
ax.annotate('d)', xy=(1, 2), xytext=(0.8, 2.15), arrowprops=arrowprops)   
ax.annotate('e)', xy=(0.14, 1.01), xytext=(0.15, 0.8), arrowprops=arrowprops)  
ax.annotate('f)', xy=(10**-1.58, 0.94), xytext=(0.033, 0.83), arrowprops=arrowprops)             
                
display(fig)
if save:
    fig.savefig('fig/SNR_for_CEW.png')
    fig.savefig('fig/SNR_for_CEW.pdf')
    print(os.path.abspath('fig/SNR_for_CEW.pdf'))
plt.close(fig)

In [ ]:
I_1 = 1
mu_1 = 1
I_2 = logspace(-1.6, 1.6, 5002)

fig, ax = plt.subplots(1, 1, figsize=(5, 2.8))
fig.set_tight_layout(True)

for k, mu_2 in enumerate([0.14, 0.25, (sqrt(2)-1), .71, 1][::-1]):
    SNR_sum = (I_1*mu_1 + I_2*mu_2)**2/(I_1 + I_2)
    ax.semilogx(I_2, SNR_sum, ls=('-' if k%2 == 0 else '--'),
                 label=(f'$\kappa_2$ = {mu_2:.2f}$\,\kappa_1$' if mu_2 != 1 else f'$\kappa_2$ = $\kappa_1$')) # /(mu_2**2+mu_1**2)
                    
mu_2 = 0.5  
SNR_sum = (I_1*mu_1 + I_2*mu_2)**2/(I_1 + I_2)
ax.semilogx(I_2, SNR_sum, ls=':', color='black',
             label=(f'$\kappa_2$ = {mu_2:.2f}$\,\kappa_1$' if mu_2 != 1 else f'$\kappa_2$ = $\kappa_1$')) # /(mu_2**2+mu_1**2)
                          

ax.set_ylabel('SNR$_{1+2}(u_a)$')
ax.set_xlabel('DEW weight $I_2/I_1$')
ax.grid(which='both')
ax.legend()#loc='lower left')
ax.set_xscale('log')#, subsx=(2, 4, 6, 8))
ax.set_ylim(0.3, 3.5)          
ticks = (0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6, 10, 30)
ax.set_xticks(ticks)
ax.set_xticklabels(['{:.4g}'.format(x) for x in ticks])
ax.set_xlim(I_2.min(), I_2.max())
                
display(fig)
if save:
    fig.savefig('fig/SNR_for_DEW.png')
    fig.savefig('fig/SNR_for_DEW.pdf')
    print(os.path.abspath('fig/SNR_for_DEW.pdf'))
plt.close(fig)

In [ ]:
def DEW_equal_SNR_point(mu_2):
    return (1-2*mu_2)/mu_2**2

print(DEW_equal_SNR_point(0.25), DEW_equal_SNR_point(0.14))

# 3.5 SNR dependency for different energy weightings, fig. 3 (bottom)

In [ ]:
I_1 = 1
I_2 = 1
mu_1 = 1
mu_2 = logspace(-1.1, 0.02, 5000)

fig, ax = plt.subplots(1, 1, figsize=(5, 2.7))
fig.set_tight_layout(True)
SNR_sum = (I_1*mu_1 + I_2*mu_2**2)**2/(I_1 + I_2*mu_2**2)
ax.semilogx(mu_2, SNR_sum, label='optimal CEW')

SNR_sum = fmax((I_1*mu_1 + I_2*mu_2)**2/(I_1 + I_2), (I_1*mu_1)**2/(I_1))
ax.semilogx(mu_2, SNR_sum, label='optimal DEW')

SNR_sum = (I_1*mu_1 + I_2*mu_2)**2/(I_1 + I_2)
ax.semilogx(mu_2, SNR_sum, label='no energy weighting')

SNR_sum = (I_1*mu_1 + 2*I_2*mu_2)**2/(I_1 + 2*I_2)*2/3
ax.semilogx(mu_2, SNR_sum, label='$\propto E$ DEW weighting', ls=':')

                    
ax.set_ylabel('SNR$_{1+2}(u_a)$')
ax.set_xlabel(r'SDS$_2$/SDS$_1$')
ax.grid(which='both')
ax.set_xscale('log')#, subsx=(2, 4, 6, 8))

ticks = (0.1, 0.3, 0.6, 1)
ax.set_xticks(ticks)
ax.set_xticklabels(['{:.4g}'.format(x) for x in ticks])
ax.set_xlim(mu_2.min(), mu_2.max())
ax.set_ylim(0., 2.3)
ax.legend()
          
if 10:
    arrowprops = dict(width=0.5, headwidth=4, headlength=4, color='black', shrink=0.02)

    ax.annotate('a)', xy=(0.14, 0.63), xytext=(0.11, 0.38), arrowprops=arrowprops)  
    ax.annotate('b)', xy=(0.25, 0.78), xytext=(0.2, 0.51), arrowprops=arrowprops)   
    ax.annotate('c)', xy=(0.41, 1), xytext=(0.32, 0.68), arrowprops=arrowprops)  
    ax.annotate('d)', xy=(1, 2), xytext=(0.9, 1.5), arrowprops=arrowprops)    
    ax.annotate('e), f)', xy=(0.14, 1), xytext=(0.101, 0.78), arrowprops=arrowprops)              
                
display(fig)
if save:
    fig.savefig('fig/SNR_for_weightings.png')
    fig.savefig('fig/SNR_for_weightings.pdf')
plt.close(fig)


# 4.1 Superposition Example: table 3

In [ ]:
mu_1 = 1
I_1 = 1
I_2 = 1

print(f'using I_1 = {I_1:.4g}, I_2 = {I_2:.4g}, mu_1 = {mu_1:.4g}\n')

for mu_2 in (0.14, 0.25, sqrt(2)-1, 1):
    SNR_2 = (I_2*mu_2)**2/(I_2)
    SNR_sum = (I_1*mu_1 + I_2*mu_2)**2/(I_1 + I_2)
    print(f'mu_2: {mu_2:.3f}, SNR_2: {SNR_2:.4f}, SNR_sum: {SNR_sum:.2f}')

mu_2 = 0.14
for WT, w_2 in zip(('CEW', 'DEW'), (0.14, 0.)):
    SNR_2 = (I_2*mu_2)**2/(I_2)
    SNR_sum = (I_1*mu_1 + I_2*mu_2*w_2)**2/(I_1 + I_2*w_2**2)
    print(f'{WT}:: mu_2: {mu_2:.2f}, SNR_2: {SNR_2:.3f}, SNR_sum: {SNR_sum:.3f}')

    

In [ ]:
plt.close('all')

# demonstrate argument from section 2.4

Higher blurring of an indirect detector leads to lower temporal noise standard deviation.

This simulation is simplified but essentially correct.

The computation uses eq. (5) and (6)

In [ ]:
from scipy import ndimage

def simulate_indirect_detection(xrays, blur_std):
    xrays_blurred = zeros_like(xrays)
    if blur_std == inf:
        for k in range(xrays.shape[0]):
            xrays_blurred[k] = xrays[k].mean()
    else:
        for k in range(xrays.shape[0]):
            xrays_blurred[k] = ndimage.gaussian_filter(xrays[k], blur_std)

    detected = random.poisson(xrays_blurred*conversion)

    detected_std = detected.std(axis=0).mean()
    return detected, detected_std

intensity = 500
xrays = random.poisson(intensity, size=(500, 500, 20))

def MTF_gauss(coords, sigma):
    if sigma == inf:
        mtf = zeros_like(coords)
        mtf[0, 0] = 1
        return mtf
    else:
        return exp(-2*pi**2*coords**2*sigma**2)
u_1D = linspace(0., 0.5, 200)
u_2D = sqrt(u_1D[newaxis, :]**2 + u_1D[:, newaxis]**2)

for conversion in (2, 20, 200):
    print(f'using conversion = {conversion}')
    for blur_std in (0., 1., 2., 4., inf):
        detected, detected_std = simulate_indirect_detection(xrays, blur_std)
        MTF = MTF_gauss(u_2D, blur_std)
        tsd = sqrt( (MTF**2*conversion**2 + conversion).mean()*intensity)
        print(f'blur std: {blur_std:.1f} px, temporal standard deviation, simulated: {detected_std:.4g}, computed: {tsd:.4g}')
    print()    
    